## Predicting Property Maintenance Fines

This project is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. Project aim is to predict whether a given blight ticket will be paid on time.

All data has been provided through the [Detroit Open Data Portal](https://data.detroitmi.gov/).
___

Each row in these data corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions**

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric is the Area Under the ROC Curve (AUC). 

___

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV

In [2]:
def train_data():
    train = pd.read_csv('train.csv',encoding = 'ISO-8859-1' ,low_memory=False)
    train = train.fillna(value=-1)
    train = train[train.compliance != -1]
    
    return train

print(train_data().shape)
train_data().describe()

(159880, 34)


,ticket_id,violation_street_number,violation_zip_code,mailing_address_str_number,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,payment_amount,balance_due,compliance
count,159880.000000,1.598800e+05,159880.0,1.598800e+05,159880.000000,159880.0,159880.0,159880.000000,159880.000000,159880.0,159880.000000,159880.000000,159880.000000,159880.000000
mean,150453.858794,1.071316e+04,-1.0,8.987563e+03,357.035295,20.0,10.0,33.651512,0.195959,0.0,420.650218,72.206537,348.263284,0.072536
std,77224.650876,3.623159e+04,0.0,3.630160e+04,675.655580,0.0,0.0,67.692916,4.290344,0.0,742.555062,259.849545,729.295700,0.259374
min,18645.000000,0.000000e+00,-1.0,-1.000000e+00,0.000000,20.0,10.0,0.000000,0.000000,0.0,0.000000,0.000000,-7750.000000,0.000000
25%,83370.750000,4.920000e+03,-1.0,4.770000e+02,200.000000,20.0,10.0,10.000000,0.000000,0.0,250.000000,0.000000,85.000000,0.000000
50%,149777.500000,1.039800e+04,-1.0,2.315000e+03,250.000000,20.0,10.0,25.000000,0.000000,0.0,305.000000,0.000000,305.000000,0.000000
75%,217480.250000,1.578325e+04,-1.0,1.267600e+04,250.000000,20.0,10.0,25.000000,0.000000,0.0,305.000000,0.000000,305.000000,0.000000
max,299363.000000,1.415411e+07,-1.0,5.111345e+06,10000.000000,20.0,10.0,1000.000000,350.000000,0.0,11030.000000,11030.000000,11030.000000,1.000000


In [3]:
def split_data():
    data=train_data()
    name=['fine_amount','admin_fee','state_fee','late_fee',
          'discount_amount','clean_up_cost','judgment_amount']
    X,y = data[name],data['compliance']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    
    return X_train, X_test, y_train, y_test

In [4]:
def test_data():
    return pd.read_csv('test.csv')
print(test_data().shape)
test_data().describe()

(61001, 27)


/home/ankit/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:4291: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,ticket_id,violation_street_number,non_us_str_code,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount
count,61001.000000,6.100100e+04,0.0,61001.000000,61001.0,61001.0,61001.000000,61001.000000,61001.000000,61001.000000
mean,331724.532811,1.256638e+04,NaN,272.714185,20.0,10.0,25.116219,0.239340,20.649711,347.895541
std,25434.932141,1.414373e+05,NaN,360.101855,0.0,0.0,36.310155,3.245894,242.375180,460.058043
min,284932.000000,-1.512600e+04,NaN,0.000000,20.0,10.0,0.000000,0.000000,0.000000,0.000000
25%,310111.000000,6.008000e+03,NaN,50.000000,20.0,10.0,5.000000,0.000000,0.000000,85.000000
50%,332251.000000,1.213400e+04,NaN,200.000000,20.0,10.0,10.000000,0.000000,0.000000,250.000000
75%,353031.000000,1.716500e+04,NaN,250.000000,20.0,10.0,25.000000,0.000000,0.000000,305.000000
max,376698.000000,2.010611e+07,NaN,10000.000000,20.0,10.0,1000.000000,250.000000,15309.000000,15558.800000


In [5]:
def choose_para(classifier,parameters,X_train, y_train):
    grid_clf_ = GridSearchCV(classifier, param_grid = parameters,scoring = 'roc_auc')
                .fit(X_train, y_train)
    
    return grid_clf_.best_params_

In [6]:
def blight_model():
    test = test_data()
    X_train, X_test, y_train, y_test = split_data()
    
    name=['fine_amount','admin_fee','state_fee','late_fee','discount_amount',
          'clean_up_cost','judgment_amount']
    clf = RandomForestClassifier()
    grid_values = {'n_estimators': [5,8,10,12],'max_depth' : [4,5,6]}
    
    para=choose_para(clf,grid_values,X_train, y_train)
    optimized_clf = RandomForestClassifier(n_estimators= para['n_estimators'],
                                           max_depth = para['n_estimators'])
                    .fit(X_train, y_train)
    
    
    
    print(optimized_clf.score(X_test,y_test))
    
    x=optimized_clf.predict_proba(test[name])

    return pd.Series(data=x[:,1],index = test['ticket_id'])
blight_model()

0.9345759319489617


ticket_id
284932    0.062633
285362    0.025995
285361    0.064098
285338    0.062633
285346    0.064098
285345    0.062633
285347    0.056825
285342    0.326389
285530    0.025995
284989    0.025995
285344    0.056825
285343    0.025995
285340    0.025995
285341    0.056825
285349    0.064098
285348    0.062633
284991    0.025995
285532    0.025995
285406    0.025995
285001    0.025995
285006    0.025995
285405    0.025995
285337    0.025995
285496    0.056825
285497    0.062633
285378    0.025995
285589    0.025995
285585    0.062633
285501    0.064098
285581    0.025995
            ...   
376367    0.025995
376366    0.035920
376362    0.035920
376363    0.062633
376365    0.025995
376364    0.035920
376228    0.035920
376265    0.035920
376286    0.312891
376320    0.035920
376314    0.035920
376327    0.312891
376385    0.312891
376435    0.426692
376370    0.312891
376434    0.056825
376459    0.064098
376478    0.010055
376473    0.035920
376484    0.025995
376482    0.025995
37